## 1. Import libraries and data

In [14]:
import pandas as pd

predicted_damage_amount = pd.read_csv("./data/exported/predicted_damage_amount.csv")
predicted_damage_incident = pd.read_csv("./data/exported/predicted_damage_incident.csv")
predicted_revenue = pd.read_csv("./data/exported/predicted_revenue.csv")

## 2. Merge 3 exported dataframes into 1

In [15]:
merged_df = pd.merge(predicted_damage_amount, predicted_damage_incident, on="applicant_id")
merged_df = pd.merge(merged_df, predicted_revenue, on="applicant_id")

### 2.1 Fix mistake in DataFrame

The models I used to predict the `predicted_damage_amount` and `predicted_damage_incident` do not line up completely.

Sometimes the `predicted_damage_incident` == 0, but the `predicted_damage_amount` is not 0 (but it should be). For example here:

In [16]:
merged_df[merged_df["predicted_damage_incident"] == 0].head(5)

,applicant_id,predicted_damage_amount,predicted_damage_incident,predicted_revenue
0,0,188.492,0,2112.740231
1,1,159.362,0,2224.849003
2,2,255.394,0,-20.213090
3,3,0.000,0,1690.207478
4,4,240.656,0,1566.944980


#### **Find out how many it got wrong**

select from df the columns where `predicted_damage_incident` is 0, but `predicted_damage_amount` is NOT 0




In [17]:
selected_columns = merged_df[
    (merged_df['predicted_damage_incident'] == 0) 
    & (merged_df['predicted_damage_amount'] != 0)]

In [18]:
selected_columns.shape

(346, 4)

it got 327 wrong:
- 327 clients got marked with 0 for predicted_damage_inc,
- but they have a predicted damage amount

#### **Manually fix the mistake**


- If we set the `predicted_damage_incident` to 1, if the `predicted_damage_amount` > $0.00, we get a ratio of approx. 68.4%
  - 0 = 158
  - 1 = 342
  - `342` / (`342`+`158`) = 0.684
- If we set the `predicted_damage_amount` to $0.00, if the `predicted_damage_incident` == 0, we get a ratio of approx. 3% 
  - 0 = 485
  - 1 = 15
  - `15` / (`15`+`485`) = 0.03


**Solution**
- 1st approach : 68.4% of applicants **will damage** the room.
- 2nd approach : 3% of applicants **will not damage** the room

The training data had a ratio of approx. 30%, and the 2nd approach is closer, and we will assume that our best applicants will not damage the room

In [19]:
merged_df.loc[merged_df["predicted_damage_incident"] == 0, "predicted_damage_amount"] = 0
merged_df[merged_df["predicted_damage_incident"] == 0].head(10)

,applicant_id,predicted_damage_amount,predicted_damage_incident,predicted_revenue
0,0,0.0,0,2112.740231
1,1,0.0,0,2224.849003
2,2,0.0,0,-20.213090
3,3,0.0,0,1690.207478
4,4,0.0,0,1566.944980
5,5,0.0,0,1551.466308
6,6,0.0,0,1570.239335
7,7,0.0,0,2309.345916
8,8,0.0,0,3326.684684
9,9,0.0,0,1682.716106


In [20]:
merged_df["predicted_damage_incident"].value_counts()

predicted_damage_incident
0    475
1     25
Name: count, dtype: int64

## 3. Calculate predicted profit per guest

`predicted_profit = predicted_revenue - predicted_damage_amount`

In [21]:
merged_df["predicted_profit"] = merged_df["predicted_revenue"] - merged_df["predicted_damage_amount"]

In [22]:
with_profit = merged_df.sort_values(by="predicted_profit", ascending=False)

In [23]:
with_profit.head()

,applicant_id,predicted_damage_amount,predicted_damage_incident,predicted_revenue,predicted_profit
100,100,0.0,0,6132.511230,6132.511230
256,256,0.0,0,5683.381907,5683.381907
431,431,0.0,0,4929.571004,4929.571004
45,45,0.0,0,4549.817353,4549.817353
199,199,0.0,0,4439.398954,4439.398954


## 04. Check if my results are even possible

The predicted profits seem very high (several thousands of dollars)

### **Checking the highest `profit_last_am` in train_V2.csv**

In [24]:
train = pd.read_csv("./data/train_V2.csv")

train[["profit_last_am"]] \
    .sort_values(by="profit_last_am", ascending=False) \
    .head(10)

,profit_last_am
3763,150537.0
866,56086.0
2212,53989.0
1563,39328.0
2493,37055.5
3593,34814.5
4818,27807.0
3984,26764.5
4512,26721.0
1986,25257.0


`.describe()`

In [25]:
train[["profit_last_am"]].describe()

,profit_last_am
count,4947.000000
mean,696.057712
std,3051.119275
min,0.000000
25%,0.000000
50%,52.000000
75%,810.000000
max,150537.000000


## 5. Export to CSV

In [26]:
merged_df.to_csv("./data/exported/merged.csv", index=False)